In [1]:
import os
import csv
import pandas as pd

In [2]:
#Download the CA & AZ HMDA Dataset

if not os.path.exists("../input_data/state_AZ-CA_mini.csv"):
    hmda = pd.read_csv('../../FIN 388/FinTech-Capstone-2023/input_data/state_AZ-CA.csv',low_memory=False,header=0)
    hmda.sample(50000).to_csv("../../FIN 388/FinTech-Capstone-2023/input_data/state_AZ-CA_mini.csv",index=False)


In [3]:
#Viewing the dataset
miniCA_AZ = pd.read_csv("../../FIN 388/FinTech-Capstone-2023/input_data/state_AZ-CA_mini.csv")
miniCA_AZ

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2021,549300HW662MN1WU8550,31084,CA,6037.0,6.037601e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3321,98.43,80000,121,595,1134,68
1,2021,5493001GDRY0EL7VG372,20940,CA,6025.0,6.025011e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,NaN,NaN,NaN,12677,80.15,56200,175,2613,3960,18
2,2021,549300FGXN1K3HLB1R50,37100,CA,6111.0,6.111001e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3806,18.81,98800,95,1009,1400,44
3,2021,7H6GLXDRUGQFU57RNE97,41940,CA,6085.0,6.085508e+09,NC,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,7001,52.41,148700,158,1626,2143,58
4,2021,549300XY701IELCE5Q08,46060,AZ,4019.0,4.019004e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,9873,43.45,68600,133,2349,3230,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2021,B4TYDEB6GKMZO031MB27,44700,CA,6077.0,6.077004e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,7823,68.59,74000,135,1937,2546,14
49996,2021,549300AG64NHILB7ZP05,38060,AZ,4013.0,4.013422e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,...,NaN,NaN,NaN,5816,26.10,79000,166,1559,2229,23
49997,2021,549300DCGBXW5FJMV921,36084,CA,6013.0,6.013321e+09,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,6813,33.74,125600,139,2344,2783,36
49998,2021,549300MGPZBLQDIL7538,46060,AZ,4019.0,4.019005e+09,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,3398,39.05,68600,107,855,1323,37


In [4]:
import cenpy
import segregation
import contextily
%matplotlib inline

ImportError: Geopandas is required to do spatial operations, and must be installed in order to use the cenpy product API. For directions on how to install geopandas, consult https://geopandas.org/install.html. Ensure that all of the dependencies, including rtree, are installed. Normally, installing geopandas through the  Anaconda Software Distribution (https://repo.continuum.io) in the "conda-forge" software channel will work.

In [5]:
#Downloading ACS data
acs = cenpy.products.ACS(2019)
acs.variables

acs.variables.to_csv("../../FIN 388/FinTech-Capstone-2023/input_data/2019cenpyvariablenames.csv")
acs.tables.to_csv("../../FIN 388/FinTech-Capstone-2023/input_data/2019cenpyvariablenamestable.csv")

NameError: name 'cenpy' is not defined

In [ ]:
az = acs.from_state('AZ')
az

In [ ]:
az = acs.from_state('AZ', variables =['B19001A_001E','B01003_001E'])
# az.shape # of tracts

## 'B02001_001E'is the RACE table for variable, 
    # but it counts for the total number of people recorded for race,
    # therefore, it has the same number from variable 'B01003_001E'recording total population in the region

#renaming the census variable for merging purposes
az.rename(columns={'GEOID':'census_tract',
                   'B19001A_001E':'Total Household Income',
                   'B01003_001E':'Total Population'}, inplace=True)
az

In [ ]:
az = acs.from_state('AZ',level='tract',variables=['B01002H_001E'])
# az.shape # of tracts

# #renaming the census variable for merging purposes
# az.rename(columns={'GEOID':'census_tract'}, inplace=True)

az

In [ ]:
# Making both census tract codes match by removing the leading '0' from ACS dataset
az["census_tract"] = int(az["census_tract"])
az

In [ ]:
# Making both census tract codes match by removing the leading '0' from ACS dataset
az["census_tract"] = az["census_tract"].astype(str)
az["census_tract"] = az["census_tract"].str.lstrip('0')
az

In [ ]:
# Converting the variable back to an int (it was a string above)
az["census_tract"] = az["census_tract"].astype(int)

#merging the dataset
merge = az.merge(df1, on = "census_tract", how = "inner")

#View the merged dataset
merge.head()